In [1]:
import pandas as pd
import requests
from IPython.display import HTML, display
from bs4 import BeautifulSoup
from bs4 import Comment
import json
import html
import re
import urllib.parse
import threading
lock = threading.RLock()

url_proxy = 'https://free-proxy-list.net/anonymous-proxy.html'
response_pxy = requests.get(url_proxy)

pxySoup = BeautifulSoup(response_pxy.text, 'html.parser')

In [2]:
rowsPxy = []
headPxy = []
for tablePart in pxySoup.find_all(attrs={'id':'proxylisttable'})[0].find_all(True, recursive=False):
    if(tablePart.name == 'thead'):
        for row in tablePart.find_all(True, recursive=False):
            headPxy = [tr.text for tr in row]
    if(tablePart.name == 'tbody'):
        for row in tablePart.find_all(True, recursive=False):
            rowsPxy.append([tr.text for tr in row])

proxiesDf = pd.DataFrame(data=rowsPxy, columns=headPxy)
proxiesDf = proxiesDf[(proxiesDf['Anonymity']=='elite proxy')].sort_values(by='Https', ascending=False)
proxiesDf.head()

,IP Address,Port,Code,Country,Anonymity,Google,Https,Last Checked
99,140.227.175.231,6000,JP,Japan,elite proxy,no,yes,2 minutes ago
85,45.230.176.146,23500,BR,Brazil,elite proxy,no,yes,2 minutes ago
33,176.36.164.240,48039,UA,Ukraine,elite proxy,no,yes,1 minute ago
83,79.172.214.2,4000,HU,Hungary,elite proxy,no,yes,2 minutes ago
82,1.20.100.227,57396,TH,Thailand,elite proxy,no,yes,2 minutes ago


In [3]:
headersChrome = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
}
proxies = {
    "http": proxiesDf[proxiesDf.Https=='no'].loc[:,'IP Address'].values[0]
}
def getProxy():
    return {
        "http": proxiesDf[proxiesDf.Https=='no'].loc[:,'IP Address'].sample(1).values
    }
# headersReloadMore =  {
#     'User-Agent': headersChrome['User-Agent'],
#     'x-spf-referer': yt_start_url,
#     'X-YouTube-Client-Name': '1',
#     'X-YouTube-Client-Version': '2.20190509',
# }

In [4]:
# rawVideos = pd.DataFrame(columns=['videoId', 'titleRaw', 'shortDesc', 'publishedTimeRaw', 'viewCountRaw', 'lengthRaw'])
rawVideos = pd.read_pickle('../datasets/rawVideos.pkl')
print(len(rawVideos), 'videos')
rawVideos.head(5)

38681 videos


,videoId,titleRaw,shortDesc,publishedTimeRaw,viewCountRaw,lengthRaw
-eLgCDFKD7s,-eLgCDFKD7s,The Art of living with Leopards | Nikit Surve ...,Nikit talks about wildlife and his work within...,1 week ago,388 views,16:46
nqSXwWUwpRM,nqSXwWUwpRM,"Puppets, guardian angels of the culture | Pou...",Persian puppets have a reach history and backg...,1 week ago,235 views,18:58
Uq1cdT4MBDc,Uq1cdT4MBDc,Unroll your role | Negar Javaherian | TEDxSUT,Some of us believe that our roles are carved i...,1 week ago,234 views,19:13
DF4ipJE_t_A,DF4ipJE_t_A,Paisajes invisibles | Francisco García & Manue...,Fran y Manuel nos ofrecen una mirada diferente...,1 week ago,200 views,12:04
u2eRC-IXkgQ,u2eRC-IXkgQ,Özgürlüğünüz Hayatınızın Temelinde Olsun | Mü...,"""Her hayır, her engel, her reddedilme benim iç...",1 week ago,212 views,15:36


In [5]:
def getFirstKey(responseText):
    return re.findall(r'\[{"nextContinuationData":{"continuation":"([^"]+)',responseText)[0]
def getNextRequestUrl(token):
    token = urllib.parse.quote(token)
    return 'https://www.youtube.com/browse_ajax?ctoken='+token+'&continuation='+token
def getNextKey(responseText):
    return responseText[1]['response']['continuationContents']['sectionListContinuation']['continuations'][0]['nextContinuationData']['continuation']
def getLoadMoreHeader(url):
    return {
    'User-Agent': headersChrome['User-Agent'],
    'x-spf-referer': url,
    'X-YouTube-Client-Name': '1',
    'X-YouTube-Client-Version': '2.20190509',
}

def getFirstDataJson(ytData):
    for scriptTxt in ytData.find_all(name='script'):
        if('window["ytInitialData"] = {' in scriptTxt.text):
#             print('FOUND')
            for sctLine in scriptTxt.text.splitlines():
                if('window["ytInitialData"] = {' in sctLine):
#                     print('FOUND')
                    return json.loads(re.findall(r'window\["ytInitialData"] = (.+);$', sctLine)[0])
    return False #error
                
def getVideoRawData(content):
    try:
        vcont = content['itemSectionRenderer']['contents'][0]['videoRenderer']
        videoId = vcont['videoId']
        title = vcont['title']['simpleText']
        description = vcont['descriptionSnippet']['simpleText']
        publishedTime = vcont['publishedTimeText']['simpleText']
        length = vcont['lengthText']['simpleText']
        viewCount = vcont['viewCountText']['simpleText']
        with lock:
            rawVideos.loc[videoId] = [videoId, title, description, publishedTime, viewCount, length]
            return [videoId, title, description, publishedTime, viewCount, length]
    except:
        return False

#getKeyfroQueryRender
def getQryKey(json):
    return json['contents']['twoColumnBrowseResultsRenderer']['tabs'][-1]\
    ['expandableTabRenderer']['content']['sectionListRenderer']['continuations']\
    [0]['nextContinuationData']['continuation']

In [6]:
def getPages(startUrl, currPage, pageKey):
    pageResponse = requests.get(
        url = getNextRequestUrl(pageKey),
        headers = getLoadMoreHeader(startUrl),
        proxies=getProxy()
    )
    pageData = json.loads(pageResponse.text)
    try:
        nextKey = getNextKey(pageData)
        #getVideos
        for vcontent in pageData[1]['response']['continuationContents']\
        ['sectionListContinuation']['contents']:
            getVideoRawData(vcontent)
        #loadMore
        currPage -= 1
#         print(currPage)
        if(currPage > 0):
            getPages(startUrl, currPage, nextKey)
    except:
        print('END OF '+startUrl)
        return False

def startLine(startUrl, typeQuery='last', maxPages=100):
    print('start ',startUrl)
    responseYt = requests.get(startUrl, headers=headersChrome, proxies=getProxy())
    #dataParse
    ytData = BeautifulSoup(responseYt.text, 'html.parser')
    firstJson = getFirstDataJson(ytData)
    if(firstJson==False):
        print('FIRST JSON ERROR')
#         display(ytData)
    #firstKey
    firstPageKey = False
    if(typeQuery=='search'):
        firstPageKey = getQryKey(firstJson)
    else:
        firstPageKey = getFirstKey(responseYt.text)
    #firstVideos
    if(typeQuery=='search'):
        for i in firstJson['contents']['twoColumnBrowseResultsRenderer']['tabs'][-1]\
        ['expandableTabRenderer']['content']['sectionListRenderer']['contents']:
            getVideoRawData(i)
    else:
        for contents in firstJson['contents']['twoColumnBrowseResultsRenderer']['tabs'][1]\
        ['tabRenderer']['content']['sectionListRenderer']['contents']:
            getVideoRawData(contents)
    getPages(startUrl, maxPages, firstPageKey)
    

# startLine('https://www.youtube.com/user/TEDxTalks/videos?view=0&sort=dd&flow=list', 'last', 3)
# startLine('https://www.youtube.com/user/TEDxTalks/videos?view=0&sort=da&flow=list', 'last', 1000000)
# startLine('https://www.youtube.com/user/TEDxTalks/search?query=Innovation', 'search', 17)
# startLine('https://www.youtube.com/user/TEDxTalks/search?query=TEDxUTN', 'search', 17)
# startLine('https://www.youtube.com/user/TEDxTalks/search?query=TEDxUADE', 'search', 17)
# startLine('https://www.youtube.com/user/TEDxTalks/search?query=TEDxUCES', 'search', 17)
# startLine('https://www.youtube.com/user/TEDxTalks/search?query=TEDxCordoba', 'search', 17)

# rawVideos

In [7]:
len(rawVideos)

38681

In [8]:
from threading import Thread

def process_id(eventName, strictSearch):
    """process a single ID"""
    eventName = re.sub('@', '%40', eventName)
    try:
        if(strictSearch):
            startLine('https://www.youtube.com/user/TEDxTalks/search?query="'+eventName+'"', 'search', 200)
        else:
            startLine('https://www.youtube.com/user/TEDxTalks/search?query='+eventName+'', 'search', 200)
    except:
        print('ERROR IN', eventName)
    return True

def process_range(id_range, strictSearch, store=None):
    """process a number of ids, storing the results in a dict"""
    if store is None:
        store = {}
    curr = 1
    for id in id_range:
        print(str(curr)+' of '+str(len(id_range)))
        curr = curr+1
        with lock:
            rawVideos.to_pickle('../datasets/rawVideos.pkl')
        store[id] = process_id(id, strictSearch)
    return store

def threaded_process_range(nthreads, id_range, strictSearch):
    """process the id range in a specified number of threads"""
    store = {}
    threads = []
    # create the threads
    for i in range(nthreads):
        ids = id_range[i::nthreads]
        t = Thread(target=process_range, args=(ids, strictSearch, store))
        threads.append(t)

    # start the threads
    [ t.start() for t in threads ]
    # wait for the threads to finish
    [ t.join() for t in threads ]
    return store

In [9]:
eventsDf = pd.read_pickle('../datasets/eventsParsed.pkl')
print(len(eventsDf), 'events')
display(eventsDf.head())

TEDCat = pd.read_csv('../TEDx categories.csv')[['CAT']]
print(len(TEDCat), 'categories')
TEDCat.head(5)

TEDCat.iloc[1:5].CAT.values

12998 events


,name,eventType,date,city,country,hasWebcast,hasSpace,eventCode,shortName
32790,TEDxRiodelaPlata,Standard,"May 7, 2019",Ciudad Autónoma de Buenos Aires,Argentina,True,False,32790,RiodelaPlata
33511,TEDxEsquel,Standard,"May 4, 2019",Esquel,Argentina,True,False,33511,Esquel
31229,TEDxRafaela,Standard,"May 4, 2019",Rafaela,Argentina,True,False,31229,Rafaela
34453,TEDxBarilocheLive,TEDxLive,"April 27, 2019",San Carlos de Bariloche,Argentina,True,False,34453,Bariloche
34383,TEDxRosarioLive,TEDxLive,"April 18, 2019",Rosario,Argentina,True,False,34383,Rosario


472 categories


array(['Addiction', 'Adventure', 'Advertising', 'Africa'], dtype=object)

In [10]:
eventsDf[eventsDf.eventType != 'TEDxLive'].name.unique()

array(['TEDxRiodelaPlata', 'TEDxEsquel', 'TEDxRafaela', ...,
       'TEDxSzczecinLive', 'TEDxGoogleWroclawLive', 'TEDxWroclawCinema'],
      dtype=object)

In [13]:
names = eventsDf[eventsDf.eventType != 'TEDxLive'].name.unique()
# names = TEDCat.CAT.values
strictSearch = True
threaded_process_range(
    15, names, strictSearch
)

1 of 477
1 of 477
1 of 477
1 of 477
1 of 476
1 of 476
1 of 476
1 of 476
1 of 476
1 of 476
1 of 476
1 of 476
1 of 476
1 of 476
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxRiodelaPlata"
1 of 476
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxEsquel"
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxRafaela"
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxSantaRosa"
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxTafiViejo"
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxYouth%40AEL"
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxGualeguaychu"
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxPaseoAlameda"
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxVillaCrespoWomen"
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxParqueSanCarlos"
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxTigre"
start  https://www.youtub

END OF https://www.youtube.com/user/TEDxTalks/search?query="TEDxMarDelPlata"
3 of 476
ERROR IN TEDxCordobaSalon
6 of 476
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxYPF"
ERROR IN TEDxYPF
7 of 476
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxJoven%40CNBA"
END OF https://www.youtube.com/user/TEDxTalks/search?query="TEDxUBA"
3 of 476
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxPaseodelBosque"
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxUTN"
END OF https://www.youtube.com/user/TEDxTalks/search?query="TEDxUNLaM"
5 of 476
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxRosarioWomen"
END OF https://www.youtube.com/user/TEDxTalks/search?query="TEDxYouth%40Funes"
5 of 476
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxBancoGalicia"
ERROR IN TEDxBancoGalicia
6 of 476
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxUADEWomen"
ERROR IN TEDxUADEWomen
7 of 476
start  https:

ERROR IN TEDxOldenburg
11 of 476
END OF https://www.youtube.com/user/TEDxTalks/search?query="TEDxPuertoMadero"
10 of 476
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxBlinnCollege"
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxGigiri"
END OF https://www.youtube.com/user/TEDxTalks/search?query="TEDxPuertoMadryn"
5 of 477
ERROR IN TEDxGigiri
11 of 476
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxJakartaSalon"
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxOlavarría"
ERROR IN TEDxJakartaSalon
12 of 476
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxCincinnati"
END OF https://www.youtube.com/user/TEDxTalks/search?query="TEDxUniversidadCatólicadeCórdoba"
5 of 476
END OF https://www.youtube.com/user/TEDxTalks/search?query="TEDxPinheiros"
12 of 476
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxPESAWomen"
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxBankingUniversity"
E

ERROR IN TEDxJNJGeel
11 of 476
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxAix"
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxSecretariaDeEducacionDelEstadoDeZacatecas"
END OF https://www.youtube.com/user/TEDxTalks/search?query="TEDxMontebelluna"
13 of 476
ERROR IN TEDxSecretariaDeEducacionDelEstadoDeZacatecas
12 of 476
END OF https://www.youtube.com/user/TEDxTalks/search?query="TEDxCincinnati"
13 of 476
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxAllianceUniversity"
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxYouth%40Brambleton"
ERROR IN TEDxAllianceUniversity
13 of 476
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxLaçador"
ERROR IN TEDxYouth%40Brambleton
14 of 476
END OF https://www.youtube.com/user/TEDxTalks/search?query="TEDxPlazaIndependencia"
7 of 476
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxGatewaySalon"
ERROR INstart  TEDxGatewaySalon
14 of 476
 https://www.you

start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxSaintJeandePassySchool"
ERROR IN TEDxSaintJeandePassySchool
21 of 476
END OF https://www.youtube.com/user/TEDxTalks/search?query="TEDxYouth%40ConinesMillPond"
22 of 476
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxJiangPuPark"
ERROR IN TEDxJiangPuPark
23 of 476
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxScarsdale"
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxUnpad"
END OF https://www.youtube.com/user/TEDxTalks/search?query="TEDxScarsdale"
22 of 476
END OF https://www.youtube.com/user/TEDxTalks/search?query="TEDxPavia"
17 of 476
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxYouth%40ÇevreCollege"
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxCaen"
END OF https://www.youtube.com/user/TEDxTalks/search?query="TEDxEscuelaAmeyalli"
19 of 476
ERROR IN TEDxCaen
18 of 476
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxAvei

...

start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxMashtotsAv"
ERROR IN TEDxXaçmaz
454 of 477
ERROR IN TEDxMashtotsAv
447 of 476
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxYouth%40PotoharRd"
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxKarangahapeRoad"
ERROR INstart  https://www.youtube.com/user/TEDxTalks/search?query="TEDxSacramentoSalon"
 TEDxYouth%40PotoharRd
473 of 476
ERROR IN TEDxKarangahapeRoad
461 of 477
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxByronBay"
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxGlasshouseChristianCollege"
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxOloibiriRoad"
ERROR IN TEDxIfe
462 of 476
ERROR IN TEDxOloibiriRoad
474 of 476
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxIkoyiWomen"
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxTeAro"
ERROR IN TEDxIkoyiWomen
466 of 476
start  https://www.youtube.com/user/TEDx

start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxSCGH"
ERROR IN TEDxPatanSalon
466 of 476
ERROR IN TEDxSCGH
445 of 477
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxYouth%40TeenTalwar"
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxRampur"
ERROR IN TEDxYouth%40TeenTalwar
471 of 476
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxDenBosch"
ERROR IN TEDxRampur
456 of 476
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxKids%40Islamabad"
ERROR IN TEDxDenBosch
467 of 476
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxRockhampton"
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxUniversityofGujrat"
ERROR IN TEDxKids%40Islamabad
472 of 476
ERROR IN TEDxUniversityofGujrat
470 of 477
ERROR IN TEDxPatan
466 of 476
ERROR IN TEDxCorlaerCollegeED
458 of 477
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxAnarkali"
start  https://www.youtube.com/user/TEDxTalks/search?quer

start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxBiratnagar"
ERROR IN TEDxEQCHCH
471 of 476
ERROR IN TEDxChicuque
456 of 477
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxEmmen"
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxYouth%40DaltonDenHaag"
ERROR IN TEDxBiratnagar
456 of 477
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxIkoyiChange"
ERROR IN TEDxEmmen
467 of 477
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxWassenaar"
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxHaarlem"
ERROR INERROR IN TEDxYouth%40DaltonDenHaag
468 of 476
 TEDxIkoyiChange
472 of 476
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxUniben"
ERROR IN TEDxWassenaar
457 of 477
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxStrijp"
start  https://www.youtube.com/user/TEDxTalks/search?query="TEDxFalconRd"
ERROR IN TEDxUniben
468 of 477
start  https://www.youtube.com/user/TEDxTalks/s

ERROR IN TEDxGoogleWroclawLive


{'TEDxRafaela': True,
 'TEDxTafiViejo': True,
 'TEDxEsquel': True,
 'TEDxVillaCrespoWomen': True,
 'TEDxJNJBuenosAires': True,
 'TEDxFunes': True,
 'TEDxPuertoSanchez': True,
 'TEDxEstaciónEwald': True,
 'TEDxGualeguaychu': True,
 'TEDxTigre': True,
 'TEDxParqueSanCarlos': True,
 'TEDxRufino': True,
 'TEDxLaPlata': True,
 'TEDxSantaRosa': True,
 'TEDxSunchales': True,
 'TEDxYouth@SwornJuniorCollege': True,
 'TEDxVicenteLopez': True,
 'TEDxSanNicolas': True,
 'TEDxRioCuarto': True,
 'TEDxSanIsidroLive': True,
 'TEDxPaseoAlameda': True,
 'TEDxHCDN': True,
 'TEDxYouth@Posadas': True,
 'TEDxYouth@AEL': True,
 'TEDxUADE': True,
 'TEDxUCC': True,
 'TEDxUTDT': True,
 'TEDxPeatonalAlberdi': True,
 'TEDxLagunaSetúbal': True,
 'TEDxJoven@ValledelYacampis': True,
 'TEDxRosarioLive': True,
 'TEDxUTNCórdoba': True,
 'TEDxPuertoNorte': True,
 'TEDxRiodelaPlataSalon': True,
 'TEDxValledelYacampis': True,
 'TEDxPESA': True,
 'TEDxPinamar': True,
 'TEDxBariloche': True,
 'TEDxRosario': True,
 'TEDxJuan

In [15]:
print(len(rawVideos), 'videos')
rawVideos.head(5)

96728 videos


,videoId,titleRaw,shortDesc,publishedTimeRaw,viewCountRaw,lengthRaw
-eLgCDFKD7s,-eLgCDFKD7s,The Art of living with Leopards | Nikit Surve ...,Nikit talks about wildlife and his work within...,7 months ago,546 views,16:46
nqSXwWUwpRM,nqSXwWUwpRM,"Puppets, guardian angels of the culture | Pou...",Persian puppets have a reach history and backg...,7 months ago,407 views,18:58
Uq1cdT4MBDc,Uq1cdT4MBDc,Unroll your role | Negar Javaherian | TEDxSUT,Some of us believe that our roles are carved i...,7 months ago,"1,077 views",19:13
DF4ipJE_t_A,DF4ipJE_t_A,Paisajes invisibles | Francisco García & Manue...,Fran y Manuel nos ofrecen una mirada diferente...,7 months ago,767 views,12:04
u2eRC-IXkgQ,u2eRC-IXkgQ,Özgürlüğünüz Hayatınızın Temelinde Olsun | Mü...,"""Her hayır, her engel, her reddedilme benim iç...",7 months ago,357 views,15:36


In [16]:
rawVideos.to_pickle('../datasets/rawVideos.pkl')